# Homework 9: Authenticating Messages
## Team 5
- Miguel Angel Tornero Carrillo - A00820449
- Francisco Salgado Guízar - A01365047
- Andres Ramírez de Alba - A01720068

### HMAC

#### hmac-server.py

In [ ]:
import socket
from Cryptodome.PublicKey import RSA
from Cryptodome.Cipher import AES
from Cryptodome.Hash import HMAC, SHA256
from Cryptodome.Cipher import PKCS1_OAEP

s=socket.socket(socket.AF_INET, socket.SOCK_STREAM);
s.bind(('127.0.0.1', 1100))
s.listen(1)

server_keypair = RSA.generate(1024)
decryptor = PKCS1_OAEP.new(server_keypair)
server_pubkey = server_keypair.public_key().export_key()

def key_decrypt(bytes: bytes, key: bytes):
    ciphertext = bytes[16:]
    nonce = bytes[:16]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)

    output = cipher.decrypt(ciphertext)
    return output

while True:
    print("Waiting for connection...")
    conn, addr = s.accept()

    print("Sending server public key:\n", server_pubkey.decode())
    conn.send(server_pubkey)
    
    encrypted_session_key = conn.recv(4096)
    session_key = decryptor.decrypt(encrypted_session_key)
    print("Received session key:", session_key.hex())
    conn.send(b'ok')

    encrypted_secret_nonce = conn.recv(4096)
    secret = key_decrypt(encrypted_secret_nonce, session_key)
    print("Received client secret:", secret.decode())    
    conn.send(b'ok')

    encrypted_message_mac = conn.recv(4096)
    message_digest = key_decrypt(encrypted_message_mac, session_key)
    mac = message_digest[:32]
    message = message_digest[32:]
    print("Received message:", message.decode())
    conn.send(b'ok')

    try:
        h = HMAC.new(secret, digestmod=SHA256)
        h.update(message)
        h.verify(mac)
        print("Valid hmac signature")
    except ValueError:
        print("Not a valid hmac signature")

#### hmac-client.py

In [ ]:
import socket
from Cryptodome.PublicKey import RSA
from Cryptodome.Hash import HMAC, SHA256
from Cryptodome.Cipher import PKCS1_OAEP
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes

def key_encrypt(bytes: bytes, key: bytes) -> bytes:
    cipher = AES.new(key, AES.MODE_EAX)
    nonce = cipher.nonce
    ciphertext = cipher.encrypt(bytes)
    return nonce + ciphertext

message = "my very secret message"

secret = b'my very secret secret'
h = HMAC.new(secret, digestmod=SHA256)

session_key = get_random_bytes(16)

s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 1100))
server_pub_key_pem = s.recv(4096)

print("Received server public key:\n", server_pub_key_pem.decode())
server_pub_key = RSA.import_key(server_pub_key_pem.decode())
encryptor = PKCS1_OAEP.new(server_pub_key)

encryped_session_key = encryptor.encrypt(session_key)
print("Sending session key:", session_key.hex())
s.send(encryped_session_key)
s.recv(4096)

msg = key_encrypt(secret, session_key)
print("Sending secret:", secret.decode())
s.send(msg)
s.recv(4096)

h.update(message.encode())
mac = h.digest()
msg = key_encrypt(mac + message.encode(), session_key)
print("Sending mac signed message:", message)
s.send(msg)
s.recv(4096)

### pkcs1_15

#### signature-server.py

In [ ]:
import socket
from Cryptodome.PublicKey import RSA
from Cryptodome.Cipher import AES
from Cryptodome.Hash import SHA256
from Cryptodome.Signature import pkcs1_15
from Cryptodome.Cipher import PKCS1_OAEP

s=socket.socket(socket.AF_INET, socket.SOCK_STREAM);
s.bind(('127.0.0.1', 1100))
s.listen(1)

server_keypair = RSA.generate(1024)
decryptor = PKCS1_OAEP.new(server_keypair)
server_pubkey = server_keypair.public_key().export_key()

def key_decrypt(bytes: bytes, key: bytes):
    ciphertext = bytes[16:]
    nonce = bytes[:16]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)

    output = cipher.decrypt(ciphertext)
    return output

while True:
    print("Waiting for connection...")
    conn, addr = s.accept()

    print("Sending server public key:\n", server_pubkey.decode())
    conn.send(server_pubkey)
    
    encrypted_session_key = conn.recv(4096)
    session_key = decryptor.decrypt(encrypted_session_key)
    print("Received session key:", session_key.hex())
    conn.send(b'ok')

    encrypted_client_public_key_nonce = conn.recv(4096)
    client_public_key_pem = key_decrypt(encrypted_client_public_key_nonce, session_key)
    print("Received client public key:\n", client_public_key_pem.decode())
    client_key = RSA.import_key(client_public_key_pem)
    verifier = pkcs1_15.new(client_key)
    conn.send(b'ok')

    encrypted_message_and_signature = conn.recv(4096)
    message_and_signature = key_decrypt(encrypted_message_and_signature, session_key)
    signature = message_and_signature[:128]
    messsage = message_and_signature[128:]
    print("Received message:", messsage.decode())
    conn.send(b'ok')

    try:
        h = SHA256.new(messsage)
        verifier.verify(h, signature)
        print("Valid signature")
    except (ValueError, TypeError):
        print("Not valid signature")

#### signature-client.py

In [ ]:
import socket
from Cryptodome.PublicKey import RSA
from Cryptodome.Hash import SHA256
from Cryptodome.Signature import pkcs1_15
from Cryptodome.Cipher import PKCS1_OAEP
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes

def key_encrypt(bytes: bytes, key: bytes) -> bytes:
    cipher = AES.new(key, AES.MODE_EAX)
    nonce = cipher.nonce
    ciphertext = cipher.encrypt(bytes)
    return nonce + ciphertext

message = "my very secret message"

client_keypair = RSA.generate(1024)
client_pubkey = client_keypair.public_key().export_key()
signer = pkcs1_15.new(client_keypair)

session_key = get_random_bytes(16)

s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 1100))
server_pub_key_pem = s.recv(4096)

print("Received server public key:\n", server_pub_key_pem.decode())
server_pub_key = RSA.import_key(server_pub_key_pem.decode())
encryptor = PKCS1_OAEP.new(server_pub_key)

encryped_session_key = encryptor.encrypt(session_key)
print("Sending session key:", session_key.hex())
s.send(encryped_session_key)
s.recv(4096)

msg = key_encrypt(client_pubkey, session_key)
print("Sending encrypted public key:\n", client_pubkey.decode())
s.send(msg)
s.recv(4096)

h = SHA256.new(message.encode())
signature = signer.sign(h)
print("signature len:", len(signature))
message_and_signature = signature + message.encode()
msg = key_encrypt(message_and_signature, session_key)
print("Sending signed message:", message)
s.send(msg)
s.recv(4096)